In [1]:
import mygrad
import mygrad.functional as F
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = mygrad.value(X_train, "x_train", requires_grad=False)
X_test = mygrad.value(X_test, "x_test", requires_grad=False)
y_train = mygrad.value(y_train, "y_train", requires_grad=False)
y_test = mygrad.value(y_test, "y_test", requires_grad=False)

In [3]:
w = mygrad.value(np.random.randn(X.shape[1]) * 10**(-2), "w", requires_grad = True)
b = mygrad.value([0.0], "b", requires_grad = True)

In [4]:
lr = 0.01

matmul1 = F.matmul()
add1 = F.add()
loss = F.bce_loss_with_logits()
s = F.sigmoid()
    
def fit(num_epoch):     
    for i in range(num_epoch):        
        y = add1.forward(matmul1.forward(X_train, w), b)
        l = loss.forward(y, y_train)
        loss.backward()
        w.data -= lr * w.grad
        b.data -= lr * b.grad
        l.zero_grad()   
        
        print(f"Epoch {i}/{num_epoch}: Train Loss: {l.data}")

def predict(X, name):
    y = add1.forward(matmul1.forward(X, w), b)
    y_pred = mygrad.value(s.forward(y).data > 0.5, name, requires_grad=False)
    
    return y_pred

Creating Function of type <class 'mygrad.functional.function.Matmul'> with name matmul_0
Creating Function of type <class 'mygrad.functional.function.Add'> with name add_0
Creating Function of type <class 'mygrad.functional.function.BCELossWithLogits'> with name bce_loss_logit_0
Creating Function of type <class 'mygrad.functional.function.Sigmoid'> with name sigmoid_0


In [5]:
fit(100)    

Epoch 0/100: Train Loss: 0.6658479041706661
Epoch 1/100: Train Loss: 0.3820898449326998
Epoch 2/100: Train Loss: 0.18157335084321288
Epoch 3/100: Train Loss: 0.1199178746367421
Epoch 4/100: Train Loss: 0.10486155469737334
Epoch 5/100: Train Loss: 0.09570161855845598
Epoch 6/100: Train Loss: 0.0900073198288788
Epoch 7/100: Train Loss: 0.08608711994416479
Epoch 8/100: Train Loss: 0.08313573137936427
Epoch 9/100: Train Loss: 0.08076793854698261
Epoch 10/100: Train Loss: 0.0787786720989437
Epoch 11/100: Train Loss: 0.07705089350006909
Epoch 12/100: Train Loss: 0.07551417873995458
Epoch 13/100: Train Loss: 0.07412408367552924
Epoch 14/100: Train Loss: 0.07285129389711549
Epoch 15/100: Train Loss: 0.07167566237117436
Epoch 16/100: Train Loss: 0.07058279263047862
Epoch 17/100: Train Loss: 0.06956200633520934
Epoch 18/100: Train Loss: 0.06860509619081763
Epoch 19/100: Train Loss: 0.06770554106940148
Epoch 20/100: Train Loss: 0.06685800146022582
Epoch 21/100: Train Loss: 0.06605798912205962
Epo

In [6]:
y_train_pred = predict(X_train, "y_train_pred")
y_test_pred = predict(X_test, "y_test_pred")

In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve, matthews_corrcoef

def print_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)    

    print(f"Confusion Matrix")
    print(f"{conf_matrix}\n")
    print(f"Accuracy: {(tp + tn)/(tp + tn + fn + fp)}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")
    print(f"AUC score: {auc(fpr, tpr)}")
    print(f"MCC score: {mcc}")
    print(f"F1-Score: {(tp)/(tp+(fp+fn)/2)}\n")
    
print_metrics(y_train.data, y_train_pred.data)
print("")
print_metrics(y_test.data, y_test_pred.data)

Confusion Matrix
[[146   4]
 [  0 276]]

Accuracy: 0.9906103286384976
Precision: 0.9857142857142858
Recall: 1.0
AUC score: 0.9866666666666667
MCC score: 0.9795042477848535
F1-Score: 0.9928057553956835


Confusion Matrix
[[60  2]
 [ 2 79]]

Accuracy: 0.972027972027972
Precision: 0.9753086419753086
Recall: 0.9753086419753086
AUC score: 0.9715252887295898
MCC score: 0.9430505774591796
F1-Score: 0.9753086419753086

